Pose Estimation Evaluation 

- SLEAP model evaluation
- 2D predictions confidence scores 
- Reprojection errors
- Missing predictions
- Joint angle statistics and distributions
- 3D animation of the pose

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from beneuro_pose_estimation import params
import beneuro_pose_estimation.evaluation as eval
from beneuro_pose_estimation.config import _load_config

config = _load_config()
# Set plot style
plt.style.use('seaborn')
sns.set_palette('husl')

## Configuration - Set session name

In [ ]:
SESSION_NAME = "M062_2025_03_21_14_00"  # Change this to your session name
TEST_NAME = "test_1"  
# TEST_NAME = None # None for full session evaluation

In [ ]:
# ===================Setup paths=======================

animal = SESSION_NAME.split("_")[0]
if TEST_NAME is not None:
    test_dir = config.predictions3D / animal / SESSION_NAME / f"{SESSION_NAME }_pose_estimation"/ "tests" / TEST_NAME 
else:
    test_dir = config.predictions3D / animal / SESSION_NAME / f"{SESSION_NAME }_pose_estimation"
csv_path = test_dir / f"{SESSION_NAME}_3dpts_angles.csv"

## Results Summary - 3D Animation, angles, missing frames

In [ ]:
# ================Frame selection=====================
# If evaluation is run on the full seession and frame range is not specified, it will be enforced to 0 - 100

output_dir = test_dir # None to not save the animation
start_frame = None
end_frame = None


if start_frame is None or end_frame is None and test_dir is None:
    start_frame = 0
    end_frame = 100


#### 3D Animation

In [ ]:
anim = eval.create_3d_animation_from_csv(
    csv_filepath=str(csv_path),
    output_dir=str(test_dir),
    start_frame=start_frame ,  
    end_frame= end_frame,    
    fps=30,
    rotation_angle=180.0, rotation_axis="z"

)

# Display the animation
from IPython.display import HTML
HTML(anim.to_jshtml())

#### Joint Angles

In [ ]:
fields = [
    "right_knee_angle",
    "left_knee_angle", 
    "right_elbow_angle", 
    "left_elbow_angle"
    ] 
# fields =  None #for all angles
eval.plot_angles(csv_path, fields = fields, frame_start=start_frame, frame_end=end_frame)

#### Missing Frames 

In [ ]:
missing_frames = eval.compute_keypoint_missing_frame_stats(csv_path)

## 2D Predictions Evaluation

### Model information - independent of test session

In [ ]:
# Choose the camera for evaluation
camera_name = "Camera_Front_Left"


In [ ]:
#  ===================Setup paths=======================
predictions2D_path = test_dir / camera_name / f"{SESSION_NAME}_{camera_name}.slp.predictions.slp"
# predictions2D_path = test_dir / f"{SESSION_NAME}_{camera_name}.slp.predictions.slp"
model_config_path = eval.get_model_path_from_slp(predictions2D_path)
model_dir = model_config_path.parent

#### Training Details

In [ ]:

eval.summarize_model_config(model_config_path)

#### Training Evaluation



Definitions
Distance Metrics
- **Localization error**  
  - Pixel distance between each predicted keypoint and its ground truth  
  - Lower values are better (0 px = perfect)

- **Object Keypoint Similarity (OKS)**  
  - Score in [0, 1], higher is better  
  - Converts per-keypoint distances into a normalized similarity w.r.t. object scale  
  - Object scale comes from the annotation bounding box (verify your boxes include occluded points)

Visualisation Metrics
- **Precision & Recall**  
  - **Precision**: fraction of predicted keypoints that are correct  
    (high precision ⇒ few false positives)  
  - **Recall**: fraction of true keypoints that are detected  
    (high recall ⇒ few false negatives)  
  - We want precision to stay high even as recall increases

---

- **mAP (mean Average Precision)**  
  - ∈ [0, 1], higher is better  
  - Area under the precision–recall curve, then averaged across OKS thresholds

- **mAR (mean Average Recall)**  
  - ∈ [0, 1], higher is better  
  - Average recall value across all OKS thresholds


In [ ]:
eval.plot_model_metrics(model_dir, split = "val", definitions = False)

### 2D Prediction Scores on test session

#### Confidence Scores 

In [ ]:
eval.visualize_confidence_scores(SESSION_NAME, test_dir)

## Triangulation Evaluation

#### Triangulation parameters - right now, just displaying the CURRENT values in params.py

**Definitions**

- **scale_smooth**  
  The weight of the temporal smoothing term in the loss function (default: 4).

  GPT: Number of frames over which to apply a smoothing filter to the object-scale estimates. A larger value yields a smoother, more stable scale over time at the cost of temporal lag.

- **scale_length**  
  The weight of the length constraints in the loss function (default: 2).

  GPT: Window size (in frames) used to compute a robust “typical” limb length from high-confidence observations. Helps enforce consistent bone lengths when data are strong.

- **scale_length_weak**  
  The weight of the weak length constraints in the loss function (default: 0.5).

  GPT: Same as `scale_length`, but applied under “weak” triangulation (fewer cameras/keypoints). Usually smaller to avoid over-smoothing scarce data.

- **reproj_error_threshold**  
  A threshold for determining which points are not suitable for triangulation (default: 15).

- **reproj_loss**  
  The loss function for the reprojection loss (default: `soft_l1`).  
  See `scipy.optimize.least_squares` for additional options.

  - `l2` — squared Euclidean distance (sensitive to outliers)  
  - `l1` — absolute distance (more robust to outliers)

- **n_deriv_smooth**  
  The order of derivative to smooth in the temporal filtering (default: 1).

  GPT: Order of finite-difference smoothing applied to the 3D trajectories (e.g. `2` = second derivative / acceleration). Higher values remove more jitter but can over-smooth rapid movements.
        
- **ransac**  
Whether to perform RANSAC triangulation (outlier view rejection based on reprojection error):  
  - `true`  — randomly sample projections to find a consensus inlier set  
  - `false` — use all available views (faster but less robust to bad detections)

| Parameter                 | Default / Typical | Notes                                             |
|---------------------------|-------------------|---------------------------------------------------|
| **scale_smooth**          | 5                 | Smoother scale vs. responsiveness                 |
| **scale_length**          | 4–8               | Window over which to estimate bone lengths        |
| **scale_length_weak**     | 1–4               | Shorter window when views are scarce              |
| **reproj_error_threshold**| 5–15              | Pixel-error scale for loss robustification        |
| **reproj_loss**           | `soft_l1`         | Balances sensitivity (L2) vs. outlier tolerance   |
| **n_deriv_smooth**        | 1–2               | Smooth velocity or acceleration                   |
| **ransac**                | `False` or `True` | Use RANSAC if you have gross outliers             |


In [ ]:
params.triangulation_params

#### Reprojection Error Analysis

In [ ]:
errors = eval.get_reprojection_errors(SESSION_NAME, test_dir, print_stats = True)

In [ ]:
eval.plot_reprojection_errors(SESSION_NAME, test_dir)

In [ ]:
fig_cam, fig_kp = eval.plot_reprojection_error_histograms(SESSION_NAME, test_dir)

In [ ]:
eval.plot_reprojection_error_per_camera(SESSION_NAME, test_dir)

In [ ]:
fig = eval.plot_reprojection_error_per_keypoint(SESSION_NAME, test_dir)

## Joint Angles Analysis

In [ ]:
start_frame = 400
end_frame = 500

if start_frame is None or end_frame is None and test_dir is None:
    # If full session is tested frame range not specified, enforce range
    start_frame = 0
    end_frame = 100

In [ ]:
# fields = ["right_knee_angle","left_knee_angle", "right_elbow_angle", "left_elbow_angle"] 
fields =  None #for all angles
eval.plot_angles(csv_path, fields = fields, frame_start=start_frame, frame_end=end_frame)

In [ ]:
eval.print_angle_stats(csv_path, frame_start=start_frame, frame_end=end_frame)

In [ ]:
eval.plot_angle_histograms(csv_path, frame_start=start_frame, frame_end=end_frame)

In [ ]:
eval.print_angle_stats_table(csv_path, frame_start=start_frame, frame_end=end_frame)

In [ ]:
eval.plot_angle_velocity_histograms(csv_path, frame_start=start_frame, frame_end=end_frame)

In [ ]:
eval.plot_angle_acceleration_histograms(csv_path, frame_start=start_frame, frame_end=end_frame)

In [ ]:
eval.plot_bodypart_autocorr_spectrum(
    csv_path,
    frame_start=start_frame,
    frame_end=end_frame,
    max_lag=60,
)